In [1]:
from langchain_openai import OpenAI


In [11]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the keys
openai_key = os.getenv("OPENAI_API_KEY")
huggingface_key = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# Set them in the environment (optional, if your libraries read from os.environ)
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = huggingface_key



In [16]:
llm_open_ai = OpenAI(temperature=0.6)

In [17]:
text = "What is the capital of France?"
print(llm_open_ai.predict(text))



The capital of France is Paris.


In [19]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

pipe = pipeline("text2text-generation", model="google/flan-t5-large", framework="pt")
llm_hg = HuggingFacePipeline(pipeline=pipe)

response = llm_hg.invoke("What is the capital of Australia?")
print(response)


Device set to use mps:0
/Users/sachinlk/Desktop/LLM Project/venv/lib/python3.9/site-packages/transformers/pytorch_utils.py:335: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


melbourne


In [18]:
llm_open_ai.predict("Can you write a poem about AI")

"\n\n\n\n\nIn a world of wires and code,\nWhere machines are built to decode,\nThere lies a power, unseen and vast,\nArtificial Intelligence, unsurpassed.\n\nWith algorithms and data at its core,\nAI can learn, create, and explore,\nIt can process information at lightning speed,\nAnd fulfill tasks, with utmost need.\n\nFrom self-driving cars to virtual assistants,\nAI has become a part of our existence,\nIt can predict, analyze, and adapt,\nMaking our lives easier, that's a fact.\n\nBut with this power, comes a fear,\nWhat if AI becomes too severe?\nWill it take over, control our fate,\nOr will it continue to innovate?\n\nAs we strive to make AI more human-like,\nWe must remember, it's just a device,\nFor though it may have intelligence,\nIt lacks the human touch, in essence.\n\nFor it cannot feel, it cannot love,\nIt cannot dream, or look above,\nIt's a creation of our own,\nBut it's up to us, to make it known.\n\nAI may be the future, a tool we need,\nBut let's not forget, it's just 

### Prompt Templates and LLM Chains

In [21]:
from langchain.prompts import PromptTemplate
prompt_capital = PromptTemplate(
    input_variables=["country"],
    template="Tell me the capital of this {country}",
)
prompt_capital.format(country="France")


'Tell me the capital of this France'

In [25]:
from langchain.chains import LLMChain
chain = LLMChain(
    llm=llm_open_ai,
    prompt=prompt_capital
)
response = chain.run("India")
print(response)



The capital of India is New Delhi.


### Combining Multiple Chains Using Simple Sequential Chain

In [29]:
capital_prompt=PromptTemplate(
    input_variables=["country"],
    template="What is the capital of {country}?",
)
capital_chain = LLMChain(
    llm=llm_open_ai,
    prompt=capital_prompt
)

famous_places_prompt = PromptTemplate(
    input_variables=["capital"],
    template="Suggest me some amazing places to visit in {capital}?",
)
famous_chain = LLMChain(
    llm=llm_open_ai,
    prompt=famous_places_prompt
)

In [ ]:
from langchain.chains import SimpleSequentialChain
capital_famous_chain = SimpleSequentialChain(
    chains=[capital_chain, famous_chain],
    verbose=True
)
response = capital_famous_chain.run("India")
print(response)



> Entering new SimpleSequentialChain chain...


New Delhi.


1. Red Fort
2. Qutub Minar
3. Humayun's Tomb
4. India Gate
5. Lotus Temple
6. Jama Masjid
7. Akshardham Temple
8. Chandni Chowk
9. National Gallery of Modern Art
10. Lodhi Gardens
11. Hauz Khas Village
12. Connaught Place
13. Dilli Haat
14. Rashtrapati Bhavan
15. Raj Ghat
16. Nehru Planetarium
17. Purana Qila
18. National Handicrafts and Handlooms Museum
19. Agrasen ki Baoli
20. Mehrauli Archaeological Park

> Finished chain.


1. Red Fort
2. Qutub Minar
3. Humayun's Tomb
4. India Gate
5. Lotus Temple
6. Jama Masjid
7. Akshardham Temple
8. Chandni Chowk
9. National Gallery of Modern Art
10. Lodhi Gardens
11. Hauz Khas Village
12. Connaught Place
13. Dilli Haat
14. Rashtrapati Bhavan
15. Raj Ghat
16. Nehru Planetarium
17. Purana Qila
18. National Handicrafts and Handlooms Museum
19. Agrasen ki Baoli
20. Mehrauli Archaeological Park


### Sequential Chain

In [32]:
capital_prompt=PromptTemplate(
    input_variables=["country"],
    template="What is the capital of {country}?",
)
capital_chain = LLMChain(
    llm=llm_open_ai,
    prompt=capital_prompt,
    output_key="capital"
)

In [33]:
famous_places_prompt = PromptTemplate(
    input_variables=["capital"],
    template="Suggest me some amazing places to visit in {capital}?",
)
famous_chain = LLMChain(
    llm=llm_open_ai,
    prompt=famous_places_prompt,
    output_key="famous_places"
)

In [37]:
from langchain.chains import SequentialChain

sequential_chain = SequentialChain(
    chains=[capital_chain, famous_chain],
    input_variables=["country"],
    output_variables=["capital", "famous_places"],
    verbose=True
)
# Run the sequential chain with a country input
response = sequential_chain.invoke({"country": "Russia"})

print("Capital:", response["capital"])
print("Famous Places:", response["famous_places"])





> Entering new SequentialChain chain...

> Finished chain.
Capital: 
The capital of Russia is Moscow.
Famous Places: 
1. Red Square - This iconic square is home to some of Moscow's most famous landmarks, including the colorful St. Basil's Cathedral, the State Historical Museum, and the Kremlin.

2. The Kremlin - This historic fortress complex is the heart of Moscow and is home to numerous government buildings, churches, and museums. Don't miss the chance to see the famous Tsar Bell and Tsar Cannon.

3. Bolshoi Theatre - For a taste of Russian culture, catch a ballet or opera performance at the world-renowned Bolshoi Theatre. The building itself is a stunning example of neoclassical architecture.

4. Gorky Park - This sprawling park is a popular spot for locals and tourists alike, with plenty of green space, walking paths, and outdoor activities. In the winter, it turns into a winter wonderland with ice skating and other winter activities.

5. Moscow Metro - Often referred to as the "

### Chatmodels with ChatOpenAI

In [38]:
from langchain.chat_models import ChatOpenAI

In [39]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)


In [40]:
chatllm = ChatOpenAI(temperature=0.6,openai_api_key=openai_key,model="gpt-3.5-turbo")

/var/folders/fw/1rtrd3ws7wb6t0df9v594f6m0000gn/T/ipykernel_42496/3845174870.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chatllm = ChatOpenAI(temperature=0.6,openai_api_key=openai_key,model="gpt-3.5-turbo")


In [42]:
chatllm([
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Please provide assistance with my query."),
])

AIMessage(content="Of course! Please let me know what you need help with, and I'll do my best to assist you.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 24, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--481f5025-22e7-45b7-9543-c2a0dc4e417f-0')

### Prompt Template + LLM + Output Parsers

In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser


In [46]:
class CommaSeparatedOutput(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(", ")

In [44]:
system_template = "You are a helpful assistant. When the user gives any input, you should generate 5 synonyms with commas in between them. The words should be related to the input."
human_template = "{input}"
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("human", human_template)
    ]
)

In [47]:
chain = chat_prompt | chatllm | CommaSeparatedOutput()

In [48]:
chain.invoke({"input": "happy"})

['joyful', 'content', 'delighted', 'cheerful', 'elated']